In [4]:
import os

In [1]:
%pwd

'd:\\Angus_Issues\\NorthCarolina_CameroonChapter_AngusIssues\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\Angus_Issues\\NorthCarolina_CameroonChapter_AngusIssues'

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [17]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [20]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [21]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-03 17:49:48,849: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-03 17:49:48,862: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-03 17:49:48,872: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-03 17:49:48,875: INFO: common: created directory at: artifacts]
[2024-08-03 17:49:48,877: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-03 17:49:51,185: INFO: 3447005916: artifacts/data_ingestion/data.zip download! with following info: 
Server: nginx
Date: Sat, 03 Aug 2024 12:19:49 GMT
Content-Type: application/zip
Content-Disposition: attachment; filename="dataset.zip"
Content-Security-Policy: frame-ancestors 'self' https://colab.research.google.com https://*.googleusercontent.com https://console.paperspace.com https://deepnoteworkspace.com https://deepnote.com https://*.mybinder.org; upgrade-insecure-requests;
Content-Transfer-Encoding: binary
Referrer-Policy: origin-when-cross-origin
Set-Cookie: